In [135]:
import numpy as np
import simpy
import random
import matplotlib
import pandas as pd

In [136]:
SEED = 930
UNIFORM_INTERVAL = [4.17, 5.37]
NUM_SERVER = 2
SERVER_UTILIZATIONS_PHASE1 = [0.6, 0.7, 0.8, 0.9]
SERVER_SERVICE_MEAN_PHASE1 = [5.725,6.679,7.634,8.588]
SERVER_UTILIATION_PHASE2 = 0.8
NUMBER_OF_CUSTOMERS = 100


In [137]:
phase_1_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIZATIONS_PHASE1,"serv_mean":SERVER_SERVICE_MEAN_PHASE1}
phase_2_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIATION_PHASE2,"serv_mean":7.634}

In [313]:
class Job(object):
    def __init__(self, name, env, operator , run_):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = self.env.process(self.call())
        self.run = run_
        
    
    def call(self):
        print('%s initiated at %g' % (self.name, self.env.now))
        self.run.num_jobs_c += 1
        self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
        self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            self.run.queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.progress_job())
            print('%s is done at %g' % (self.name, self.env.now))
            self.departure_t = self.env.now
            self.run.num_jobs_c -= 1
            self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
            self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])

            
    def progress_job(self):
        duration = random.expovariate(self.run.serv_rate)
        yield self.env.timeout(duration)
        self.run.service_times.append(duration)
        self.departure_t = self.env.now
        self.run.num_jobs_c -= 1
        self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
        self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])
        

In [314]:
class Run():
    def __init__(self,arr_rate,serv_rate):
        self.seed = random.random()*425
        self.run = False
        self.arr_rate = arr_rate
        self.serv_rate = 1/serv_rate
        self.num_jobs_c = 0
        self.num_jobs_system = [[0,0,-1]]
        self.jobs=[]
        self.service_times=[]
        self.queue_w_times=[]
        self.env = simpy.Environment()
        self.operator = simpy.Resource(self.env, capacity = NUM_SERVER)
    
        


    def run_sim(self):
        random.seed(self.seed)
        self.env.process(self.jobs_generator())
        self.env.run()
        self.run = True
        self.average_num_jobs_system()
        
    def show_run(self):
        if self.run!=True:
            self.run_sim()
        
    def average_num_jobs_system(self):
        weighted_value =0
        for arr in self.num_jobs_system:
            if(arr[2]==-1):
                end_of_sim = arr[1]
                break
            weighted_value += arr[0]*(arr[2]-arr[1])
        self.avg_jobs = weighted_value / end_of_sim

    def jobs_generator(self):
        for i in range(NUMBER_OF_CUSTOMERS):
            yield self.env.timeout(random.expovariate(self.arr_rate))
            job = Job('Job %s' %(i+1), self.env, self.operator,self)
            self.jobs.append(job)  

In [315]:
class Phase():
    def __init__(self,lib,num_runs):
        self.num_server = lib["num_server"]
        self.arr_int = 1/((lib["arr_interval"][0]+lib["arr_interval"][1])/2)
        self.util = lib["util"]
        self.serv_mean = lib["serv_mean"]
        self.init_runs(num_runs)
        self.num_runs = num_runs

    def init_runs(self,num_runs):
        self.runs = []
        for util_no in range(len(self.util)):
            for i in range(num_runs):
                obj = Run(self.arr_int,self.serv_mean[util_no])
                self.runs.append(obj)

    def run_phase(self):
        for run in self.runs:
            run.run_sim()
            
        print("total run :",len(self.runs)," ")
    
    def create_df(self,util):
        
        for i in range(len(self.ens_avg[util])):
            if(i == 0):
                ens_sum_avg = [self.ens_avg[util][0]]
                ens_cum_avg = [self.ens_avg[util][0]]
            else:
                ens_sum_avg.append((ens_sum_avg[len(ens_sum_avg)-1]+self.ens_avg[util][i])) 
                ens_cum_avg.append((ens_sum_avg[len(ens_sum_avg)-1]/(i+1)))

        df_ens_avg = pd.DataFrame(self.ens_avg[util],columns=["AVG"])
        df_ens_sum_avg = pd.DataFrame(ens_sum_avg,columns=["SUM-AVG"])
        df_ens_cum_avg = pd.DataFrame(ens_cum_avg,columns=["CUM-AVG"])
        df = pd.concat([df_ens_avg,df_ens_sum_avg,df_ens_cum_avg],ignore_index=True,axis=1)

        selected_runs = self.runs[(util)*self.num_runs:(util+1)*self.num_runs]
        
        for run_no, run in enumerate(selected_runs):
            df_time_spend = pd.DataFrame(np.sum([run.service_times,run.queue_w_times],axis=0),columns=["REPLICATION-"+str(run_no)])
            print(df_time_spend)
            df = pd.concat([df,df_time_spend],ignore_index=True,axis=1)

        return df

    def ensemble_averages(self):
        self.ens_avg = [[0 for i in range(NUMBER_OF_CUSTOMERS)] for j in range(len(self.util))]
        for k in range(len(self.util)):
            for run_no in range(self.num_runs):
                for i in range(NUMBER_OF_CUSTOMERS):
                    self.ens_avg[k][i] = self.ens_avg[k][i] + (self.runs[k*self.num_runs+run_no].jobs[i].departure_t - self.runs[k*self.num_runs+run_no].jobs[i].arrival_t)
        self.ens_avg = np.divide(self.ens_avg,self.num_runs)
            

In [316]:
phase_1 = Phase(phase_1_lib,10)

In [317]:
phase_1.run_phase()

Job 1 initiated at 0.776601
Job 1 is assigned to an operator at 0.776601
Job 2 initiated at 1.12386
Job 2 is assigned to an operator at 1.12386
Job 1 is done at 2.0934
Job 2 is done at 5.04257
Job 3 initiated at 8.0035
Job 3 is assigned to an operator at 8.0035
Job 4 initiated at 9.96577
Job 4 is assigned to an operator at 9.96577
Job 5 initiated at 16.6671
Job 6 initiated at 17.0855
Job 4 is done at 19.7822
Job 5 is assigned to an operator at 19.7822
Job 3 is done at 20.7468
Job 6 is assigned to an operator at 20.7468
Job 5 is done at 22.0764
Job 6 is done at 23.3838
Job 7 initiated at 28.018
Job 7 is assigned to an operator at 28.018
Job 8 initiated at 30.7218
Job 8 is assigned to an operator at 30.7218
Job 8 is done at 33.5484
Job 7 is done at 36.572
Job 9 initiated at 39.9261
Job 9 is assigned to an operator at 39.9261
Job 9 is done at 45.0838
Job 10 initiated at 52.7294
Job 10 is assigned to an operator at 52.7294
Job 11 initiated at 56.8689
Job 11 is assigned to an operator at 56

In [318]:
phase_1.ensemble_averages()
print(len(phase_1.ens_avg),len(phase_1.ens_avg[0]))
np.mean(phase_1.ens_avg,axis=1)

4 100


array([ 7.3012242 , 11.90820993, 18.27085574, 35.41293499])

In [319]:
phase_1.ensemble_averages

<bound method Phase.ensemble_averages of <__main__.Phase object at 0x12de36e10>>

In [320]:
df = phase_1.create_df(0)
df.to_csv("df.csv")

    REPLICATION-0
0        1.316798
1        3.918710
2        9.816432
3       12.743276
4        5.409289
..            ...
95      26.692095
96       3.752197
97       0.441639
98       0.359933
99       3.961703

[100 rows x 1 columns]
    REPLICATION-1
0        1.484194
1        3.325918
2        1.133360
3        7.444407
4        4.412829
..            ...
95      12.062347
96       6.977924
97       7.476863
98       2.875462
99       5.278105

[100 rows x 1 columns]
    REPLICATION-2
0        1.016954
1       10.769971
2        8.040728
3       22.738550
4        5.541072
..            ...
95       0.022106
96       1.661503
97       1.888818
98       0.531158
99       3.079756

[100 rows x 1 columns]
    REPLICATION-3
0        0.102688
1        1.348918
2        6.162092
3       19.310489
4        5.923770
..            ...
95      17.109559
96       4.861951
97       7.966297
98       2.826126
99      24.135247

[100 rows x 1 columns]
    REPLICATION-4
0        0.589231
1   

In [321]:

total =0
runsss = phase_1.runs[:10]
for run in runsss:
    total = total + run.jobs[0].departure_t -run.jobs[0].arrival_t
print(total)

26.968738805523866


In [322]:
print(phase_1.runs[0].service_times[0],phase_1.runs[0].queue_w_times[0],phase_1.runs[0].jobs[0].departure_t-phase_1.runs[0].jobs[0].arrival_t)

1.3167983329556943 0.0 1.316798332955694
